In [69]:
import openai

In [70]:
import minsearch

In [71]:
import json

In [72]:
with open("documents.json", "rt") as f_in:
    docs_raw = json.load(f_in)

In [73]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [74]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [75]:
# keyword_fields are used for filtering
# perform search on question text and section
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

In [76]:
from openai import OpenAI
client = OpenAI()

In [77]:
def search(query):
    boost = {'question' : 3.0, 'section' : 0.5}
    results = index.search(
        query = query,
        filter_dict = {'course' : 'data-engineering-zoomcamp'},
        boost_dict = boost,
        num_results = 5
    )
    return results

In [78]:
def build_prompt(query, search_results):
    prompt_template = """ 
You are a course teaching assistant. Answer the QUESTION based on CONTEXT. 
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT doesnt contain the answer, output None.

QUESTION: {question}

CONTEXT: {context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']} \nquestion: {doc['question']} \ntext: {doc['text']}"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [79]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{'role': 'user', "content": prompt}]
    )

    return response.choices[0].message.content

In [80]:
query = "the course has already started, can I still enroll?"
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [81]:
rag(query)

'Yes, you can still enroll in the course even after it has started.'

In [84]:
query = "how do I run kafka?"
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [85]:
rag(query)

'None'